In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
import time

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'notebooks'))
from utilities2015 import *

dm = DataManager(data_dir=os.environ['GORDON_DATA_DIR'], 
                 repo_dir=os.environ['GORDON_REPO_DIR'], 
                 result_dir=os.environ['GORDON_RESULT_DIR'], 
                 labeling_dir=os.environ['GORDON_LABELING_DIR'],
                 stack='MD593',
                 section=117)

print 'reading image ...',
t = time.time()
dm._load_image(versions=['gray'])
dm._generate_kernels()
print 'done in', time.time() - t, 'seconds'

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2261: DecompressionBombWarning: Image size (203470848 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


reading image ... done in 5.67617607117 seconds


In [2]:
from skimage.util import pad

approx_bg_intensity = dm.image[10:20, 10:20].mean()
# approx_bg_intensity = 0

masked_image = dm.image.copy()
masked_image[~dm.mask] = approx_bg_intensity

# padded_image = pad(masked_image, max_kern_size, 'constant', constant_values=approx_bg_intensity)
padded_image = pad(masked_image, dm.max_kern_size, 'linear_ramp', end_values=approx_bg_intensity)

# plt.imshow(padded_image, cm.Greys_r)
# plt.show()

# display(padded_image)

In [ ]:
import time
b = time.time()

from joblib import Parallel, delayed
from scipy.signal import fftconvolve

# try:
#     features = dm.load_pipeline_result('features', 'npy')
    
# except Exception as e:

def convolve_per_proc(i):
    return fftconvolve(padded_image, dm.kernels[i], 'same').astype(np.half)

padded_filtered = Parallel(n_jobs=16)(delayed(convolve_per_proc)(i) 
                        for i in range(dm.n_kernel))

filtered = [f[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size] for f in padded_filtered]

#     features = np.empty((dm.image_height, dm.image_width, n_kernel), dtype=np.half)
#     for i in range(n_kernel):
#         features[...,i] = filtered[i]

features = np.empty((dm.n_kernel, dm.image_height, dm.image_width), dtype=np.half)
for i in range(dm.n_kernel):
    features[i, ...] = filtered[i]

del filtered

dm.save_pipeline_result(features, 'features', 'npy')

print 'gabor filtering', time.time() - b

In [17]:
# # visualize a slice of feature responses

# cropped_response = features[-1]
# plt.matshow(cropped_response, cmap=cm.coolwarm)
# plt.colorbar()
# plt.show()

In [18]:
# from skimage.exposure import rescale_intensity
# cropped_response_vis = rescale_intensity(cropped_response, out_range=(0, 255))
# cropped_response_vis[~dm.mask] = 127
# display(cropped_response_vis.astype(np.uint8))